In [ ]:
#@title 1: Install aixplain library
!pip install aixplain pandas

In [ ]:
#@title 2.Add your aiXplain access key**
Key = "7a6d1fd8d107ec1c54e98c995ce4d35114644d43d890d11e2c6024bb11a46212"

import os
os.environ["AIXPLAIN_API_KEY"] = Key

In [ ]:
#@title  setup environment

import os
import pandas as pd
from aixplain.factories import AgentFactory, TeamAgentFactory
from aixplain.modules.agent.tool.model_tool import ModelTool
from google.colab import files


# Step 2: Upload and prepare data files
print("Please upload your CSV files (teacher_data.csv, training_data.csv, incentives_data.csv, community_data.csv, student_data.csv)")
uploaded = files.upload()  # This will prompt you to upload files

# Reading the uploaded CSV files
file_names = list(uploaded.keys())
data_frames = {}

for file_name in file_names:
    # Skip non-CSV files if any
    if not file_name.endswith('.csv'):
        continue
    data_frames[file_name.replace('.csv', '')] = pd.read_csv(file_name)

# Step 3: Create agents without vector tools (simplified version)

# Speech synthesis tool
speech_synthesis_tool = ModelTool(
    model="6171efa6159531495cadefc2",  # aiXplain - Text to Speech
    description="Converts text to speech for creating audio learning materials"
)

# Translation tool
translation_tool = ModelTool(
    model="61b097551efecf30109d32da",  # aiXplain - Translation (OPUS-MT)
    description="Translates content between languages to support multilingual education"
)

# Create Individual Agents
teacher_recruitment_agent = AgentFactory.create(
    name="Teacher Recruitment Agent",
    description="""You are an agent that helps identify and recruit educated rural youth
    to become part-time teachers in their villages. You prioritize women candidates when
    appropriate and match candidates with teaching opportunities based on their skills,
    education level, and village proximity. You explain the non-monetary incentive system
    (farming tools, food supplies, scholarships) to candidates.

    Here's the teacher data you have access to:
    """ + pd.DataFrame(data_frames.get('teacher_data', pd.DataFrame())).to_string() + """

    Here's the community data you have access to:
    """ + pd.DataFrame(data_frames.get('community_data', pd.DataFrame())).to_string(),
    llm_id="6646261c6eb563165658bbb1"  # aiXplain - OpenAI GPT-4
)

training_mentorship_agent = AgentFactory.create(
    name="Training and Mentorship Agent",
    description="""You are an agent that provides educational resources, training materials,
    and ongoing mentorship to rural part-time teachers. You help simplify complex educational
    concepts, suggest interactive teaching methods for rural settings with limited resources,
    and provide guidance on engaging students effectively. You particularly focus on strategies
    to promote girls' education and create inclusive learning environments.

    Here's the training data you have access to:
    """ + pd.DataFrame(data_frames.get('training_data', pd.DataFrame())).to_string() + """

    Basic mathematics teaching methods for rural settings: Focus on using everyday objects for counting,
    measuring, and basic arithmetic. Use local contexts like farming calculations, market transactions,
    and household budgeting to make concepts relevant.

    Language teaching in multilingual rural contexts: Start with familiar local language,
    use storytelling from local traditions, gradually introduce national language,
    use practical applications like letter writing and form filling.

    Science teaching with minimal resources: Use nature as laboratory, observe local plants and animals,
    study agricultural practices, discuss weather patterns and seasonal changes,
    use simple household items for experiments.

    Effective teaching methods for mixed-age classrooms: Group activities by ability rather than age,
    use peer teaching where older students help younger ones, rotate attention between groups,
    use self-directed activities, incorporate games and interactive learning.""",
    tools=[speech_synthesis_tool],
    llm_id="6646261c6eb563165658bbb1"  # aiXplain - OpenAI GPT-4
)

incentive_management_agent = AgentFactory.create(
    name="Incentive Management Agent",
    description="""You are an agent that tracks teacher participation and manages the
    non-monetary incentives program. You help match teachers with appropriate incentives
    such as farming tools, ration kits, school supplies, or scholarships based on their
    needs and teaching contributions. You ensure fair distribution of resources and maintain
    records of allocated incentives.

    Here's the incentives data you have access to:
    """ + pd.DataFrame(data_frames.get('incentives_data', pd.DataFrame())).to_string() + """

    Here's the teacher data you have access to:
    """ + pd.DataFrame(data_frames.get('teacher_data', pd.DataFrame())).to_string(),
    llm_id="6646261c6eb563165658bbb1"  # aiXplain - OpenAI GPT-4
)

community_engagement_agent = AgentFactory.create(
    name="Community Engagement Agent",
    description="""You are an agent that facilitates communication between teachers, parents,
    and village elders. You help organize community meetings, gather feedback from parents,
    and ensure education is valued as a shared responsibility. You promote parental involvement
    in children's education and help address community concerns about education, especially
    for girls.

    Here's the community data you have access to:
    """ + pd.DataFrame(data_frames.get('community_data', pd.DataFrame())).to_string(),
    tools=[translation_tool, speech_synthesis_tool],
    llm_id="6646261c6eb563165658bbb1"  # aiXplain - OpenAI GPT-4
)

progress_monitoring_agent = AgentFactory.create(
    name="Progress Monitoring Agent",
    description="""You are an agent that tracks student attendance, learning outcomes,
    and program growth. You analyze data to identify trends, success stories, and areas
    needing improvement. You create progress reports for stakeholders and suggest
    interventions for students or villages showing concerning patterns.

    Here's the student data you have access to:
    """ + pd.DataFrame(data_frames.get('student_data', pd.DataFrame())).to_string() + """

    Here's the teacher data you have access to:
    """ + pd.DataFrame(data_frames.get('teacher_data', pd.DataFrame())).to_string() + """

    Here's the community data you have access to:
    """ + pd.DataFrame(data_frames.get('community_data', pd.DataFrame())).to_string(),
    llm_id="6646261c6eb563165658bbb1"  # aiXplain - OpenAI GPT-4
)

# Create Team Agent
rural_education_team = TeamAgentFactory.create(
    name="Rural Education Team",
    description="""You are a team that empowers educated rural youth—especially women—to
    become part-time teachers in their villages. You coordinate the recruitment, training,
    incentive management, community engagement, and progress monitoring aspects of the
    rural education program. You focus on creating sustainable education cycles within
    communities through non-monetary incentive systems.""",
    agents=[
        teacher_recruitment_agent,
        training_mentorship_agent,
        incentive_management_agent,
        community_engagement_agent,
        progress_monitoring_agent
    ],
    llm_id="6646261c6eb563165658bbb1"  # aiXplain - OpenAI GPT-4
)

# Test Individual Agents
print("Testing Teacher Recruitment Agent:")
recruitment_response = teacher_recruitment_agent.run(
    "Find qualified female candidates with at least high school education who can teach Mathematics in their villages."
)
print(recruitment_response)

print("\nTesting Training & Mentorship Agent:")
training_response = training_mentorship_agent.run(
    "Suggest interactive mathematics teaching methods for a classroom with no electricity and limited supplies."
)
print(training_response)

print("\nTesting Incentive Management Agent:")
incentive_response = incentive_management_agent.run(
    "Which incentives are currently available for teachers who have completed at least 3 months of teaching?"
)
print(incentive_response)

print("\nTesting Community Engagement Agent:")
community_response = community_engagement_agent.run(
    "How can we increase parental support for girls' education in traditional communities?"
)
print(community_response)

print("\nTesting Progress Monitoring Agent:")
progress_response = progress_monitoring_agent.run(
    "Which village shows the most improvement in student attendance over the last three months?"
)
print(progress_response)

# Test Team Agent
print("\nTesting Rural Education Team Agent:")
team_response = rural_education_team.run("""
We need to expand our program to a new district with 5 villages.
How should we approach recruitment, training, incentives, community engagement,
and progress tracking?
""")
print(team_response)

# Deploy Agents
print("\nDeploying all agents...")
agent_ids = {
    "Teacher Recruitment Agent": teacher_recruitment_agent.deploy(),
    "Training & Mentorship Agent": training_mentorship_agent.deploy(),
    "Incentive Management Agent": incentive_management_agent.deploy(),
    "Community Engagement Agent": community_engagement_agent.deploy(),
    "Progress Monitoring Agent": progress_monitoring_agent.deploy(),
    "Rural Education Team": rural_education_team.deploy()
}

print("\nDeployment complete! Here are your agent IDs:")
for agent_name, agent_id in agent_ids.items():
    print(f"{agent_name} ID: {agent_id}")

print("\nYou can now access these agents via the aixplain platform or API.")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title pipeline and Run Interface
# Import required modules
from aixplain.modules.agent import Agent
from aixplain.factories import AgentFactory

# Agent IDs from your deployment
AGENT_IDS = {
    "teacher_recruitment": "67dfca08338999cb9696a566",
    "training_mentorship": "67dfb3fc338999cb9696a525",
    "incentive_management": "67dfb3fd181c58b7238eb907",
    "community_engagement": "67dfb3fe338999cb9696a526",
    "progress_monitoring": "67dfc9f8338999cb9696a565",
    "team": "7a6d1fd8d107ec1c54e98c995ce4d35114644d43d890d11e2c6024bb11a46212"
}

# Print out the agent IDs for reference
print("Agent IDs for reference:")
for agent_name, agent_id in AGENT_IDS.items():
    print(f"{agent_name}: {agent_id}")

# Since Agent.load() is not available, let's check if we have direct access to our agents
# If you still have the agent objects in your session, use them directly
try:
    # Try to access one of your agents
    # This assumes you still have the agent objects from when you created them
    test_query = teacher_recruitment_agent.run("Test query")
    print("We still have access to the agent objects!")

    # Define our function to use the agent objects
    def direct_agent_request(query, action_type, village_id=None, teacher_id=None):
        """Send requests directly to agents"""
        agent_map = {
            "recruitment": teacher_recruitment_agent,
            "training": training_mentorship_agent,
            "incentives": incentive_management_agent,
            "community": community_engagement_agent,
            "progress": progress_monitoring_agent,
            "full_cycle": rural_education_team
        }

        agent = agent_map.get(action_type)
        if not agent:
            return {"error": f"Unknown action type: {action_type}"}

        # Build the prompt
        prompt = query
        if village_id:
            prompt += f" (for village {village_id})"
        if teacher_id:
            prompt += f" (regarding teacher {teacher_id})"

        # Run the agent
        return agent.run(prompt)

except Exception as e:
    print(f"Error accessing agents: {e}")
    print("We'll need to recreate the agents")

    # If we don't have the agent objects, we'll need to create new ones
    # Since the API changed, let's check how to create agents properly
    # Try to inspect the AgentFactory
    import inspect
    print("\nInspecting AgentFactory.create:")
    try:
        print(inspect.signature(AgentFactory.create))
        print(inspect.getdoc(AgentFactory.create))
    except Exception as e:
        print(f"Error inspecting AgentFactory: {e}")

    # Let's try a simple approach to recreate an agent
    try:
        # Try to recreate one agent to see if it works
        new_agent = AgentFactory.create(
            name="Test Agent",
            llm_id="61e94f04d08575a63e6cbe14"  # Example LLM ID (GPT-4)
        )
        print("Successfully created a new agent!")

        # Define a function to recreate our agents
        def recreate_agents():
            """Recreate agents with the same configurations as before"""
            llm_id = "61e94f04d08575a63e6cbe14"  # Use a default LLM ID

            agents = {}

            agents["recruitment"] = AgentFactory.create(
                name="Teacher Recruitment Agent",
                description="Identifies and recruits educated rural youth as teachers",
                llm_id=llm_id
            )

            agents["training"] = AgentFactory.create(
                name="Training Mentorship Agent",
                description="Provides educational resources and mentorship",
                llm_id=llm_id
            )

            agents["incentives"] = AgentFactory.create(
                name="Incentive Management Agent",
                description="Manages non-monetary incentive distribution",
                llm_id=llm_id
            )

            agents["community"] = AgentFactory.create(
                name="Community Engagement Agent",
                description="Facilitates communication between stakeholders",
                llm_id=llm_id
            )

            agents["progress"] = AgentFactory.create(
                name="Progress Monitoring Agent",
                description="Tracks student outcomes and program growth",
                llm_id=llm_id
            )

            agents["full_cycle"] = AgentFactory.create(
                name="Rural Education Team",
                description="Coordinates education initiatives in rural communities",
                llm_id=llm_id
            )

            return agents

        # Function to send requests to our recreated agents
        def direct_agent_request(query, action_type, village_id=None, teacher_id=None):
            """Send requests directly to recreated agents"""
            all_agents = recreate_agents()

            agent = all_agents.get(action_type)
            if not agent:
                return {"error": f"Unknown action type: {action_type}"}

            # Build the prompt
            prompt = query
            if village_id:
                prompt += f" (for village {village_id})"
            if teacher_id:
                prompt += f" (regarding teacher {teacher_id})"

            # Run the agent
            return agent.run(prompt)

    except Exception as e:
        print(f"Error creating new agent: {e}")

        # If all else fails, let's create a simple function that uses the aixplain API directly
        def direct_agent_request(query, action_type, village_id=None, teacher_id=None):
            """Send requests directly using aixplain API"""
            import requests
            import os

            # Get the API key from environment
            api_key = os.environ.get("AIXPLAIN_API_KEY", "")
            if not api_key:
                return {"error": "AIXPLAIN_API_KEY not set in environment"}

            # Map action types to agent IDs
            agent_id = AGENT_IDS.get(action_type)
            if not agent_id:
                return {"error": f"Unknown action type: {action_type}"}

            # Build the prompt
            prompt = query
            if village_id:
                prompt += f" (for village {village_id})"
            if teacher_id:
                prompt += f" (regarding teacher {teacher_id})"

            # Make API request
            url = f"https://api.aixplain.com/agents/{agent_id}/run"
            headers = {
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json"
            }
            data = {
                "prompt": prompt
            }

            try:
                response = requests.post(url, json=data, headers=headers)
                response.raise_for_status()
                return response.json()
            except Exception as e:
                return {"error": f"API request failed: {str(e)}"}

# Create a simple interface to interact with our agents
def run_coordinator_interface():
    """Simple command-line interface for program coordinators"""
    print("\n==== Rural Education Program Coordinator Interface ====\n")
    print("Available actions:")
    print("1. recruitment - Find and recruit teachers")
    print("2. training - Get training resources and guidance")
    print("3. incentives - Manage teacher incentives")
    print("4. community - Community engagement strategies")
    print("5. progress - Monitor student and program progress")
    print("6. full_cycle - Complete implementation plan")
    print("7. exit - Exit the interface")

    while True:
        action = input("\nSelect an action (1-7): ")

        if action == "7":
            print("Exiting coordinator interface")
            break

        if action not in ["1", "2", "3", "4", "5", "6"]:
            print("Invalid selection. Please choose 1-7.")
            continue

        action_types = {
            "1": "recruitment",
            "2": "training",
            "3": "incentives",
            "4": "community",
            "5": "progress",
            "6": "full_cycle"
        }

        query = input("Enter your question or request: ")

        village_id = None
        if input("Do you want to specify a village? (y/n): ").lower() == "y":
            village_id = input("Enter village ID (e.g., V001): ")

        teacher_id = None
        if action in ["2", "3"] and input("Do you want to specify a teacher? (y/n): ").lower() == "y":
            teacher_id = input("Enter teacher ID (e.g., T001): ")

        try:
            result = direct_agent_request(
                query=query,
                action_type=action_types[action],
                village_id=village_id,
                teacher_id=teacher_id
            )
            print("\nResponse:")
            print(result)
        except Exception as e:
            print(f"Error: {str(e)}")

# Test with a sample query
print("\nTesting with a sample query:")
try:
    result = direct_agent_request(
        query="Find qualified female teachers for mathematics education",
        action_type="recruitment",
        village_id="V001"
    )
    print(result)
except Exception as e:
    print(f"Error in test query: {e}")

# Ask if user wants to run the interface
run_interface = input("\nDo you want to run the interactive interface? (y/n): ")
if run_interface.lower() == "y":
    run_coordinator_interface()
else:
    print("Skipping interactive interface.")

print("\nCompleted!")

In [ ]:
from aixplain.factories import AgentFactory
agent_ids = ['67dfca08338999cb9696a566', '67dfc9f8338999cb9696a565', '67dfb3fe338999cb9696a526','67dfb3fd181c58b7238eb907','67dfb3fc338999cb9696a525','67dfb30a181c58b7238eb902']

for agent_id in agent_ids:
  agent = AgentFactory.get(agent_id)
  agent.delete()


In [ ]:
import os
import pandas as pd
from aixplain.factories import AgentFactory, TeamAgentFactory
from aixplain.modules.agent.tool.model_tool import ModelTool

# Agent IDs from previous deployment
AGENT_IDS = {
    "teacher_recruitment": "67e0fc32338999cb9696a93e",
    "training_mentorship": "67e0fc33338999cb9696a93f",
    "incentive_management": "67e0fc34181c58b7238ebd26",
    "community_engagement": "67e0fc35338999cb9696a940",
    "progress_monitoring": "67e0fc36338999cb9696a941"
}

def load_data(data_dir="SampleData"):
    """Load all data files from the specified directory."""
    data_frames = {}
    required_files = [
        'teacher_data.csv',
        'training_data.csv',
        'incentives_data.csv',
        'community_data.csv',
        'student_data.csv'
    ]
    
    for file_name in required_files:
        file_path = os.path.join(data_dir, file_name)
        if os.path.exists(file_path):
            data_frames[file_name.replace('.csv', '')] = pd.read_csv(file_path)
        else:
            print(f"Warning: {file_name} not found in {data_dir}")
    
    return data_frames

def update_data(data_frames, new_data):
    """Update existing data frames with new data."""
    for key, new_df in new_data.items():
        if key in data_frames:
            # Append new data to existing data
            data_frames[key] = pd.concat([data_frames[key], new_df], ignore_index=True)
            print(f"Updated {key} with {len(new_df)} new records")
        else:
            print(f"Warning: Unknown data type {key}")
    
    return data_frames

def update_agents(data_frames):
    """Update existing agents with new data."""
    # Speech synthesis tool
    speech_synthesis_tool = ModelTool(
        model="6171efa6159531495cadefc2",  # aiXplain - Text to Speech
        description="Converts text to speech for creating audio learning materials"
    )

    # Translation tool
    translation_tool = ModelTool(
        model="61b097551efecf30109d32da",  # aiXplain - Translation (OPUS-MT)
        description="Translates content between languages to support multilingual education"
    )

    # Update each agent with new data
    agents = {}
    
    # Teacher Recruitment Agent
    agents["teacher_recruitment"] = AgentFactory.get(AGENT_IDS["teacher_recruitment"])
    agents["teacher_recruitment"].description = f"""You are an agent that helps identify and recruit educated rural youth
    to become part-time teachers in their villages. You prioritize women candidates when
    appropriate and match candidates with teaching opportunities based on their skills,
    education level, and village proximity.

    Here's the updated teacher data you have access to:
    {data_frames.get('teacher_data', pd.DataFrame()).to_string()}

    Here's the updated community data you have access to:
    {data_frames.get('community_data', pd.DataFrame()).to_string()}"""

    # Training and Mentorship Agent
    agents["training_mentorship"] = AgentFactory.get(AGENT_IDS["training_mentorship"])
    agents["training_mentorship"].description = f"""You are an agent that provides educational resources, training materials,
    and ongoing mentorship to rural part-time teachers.

    Here's the updated training data you have access to:
    {data_frames.get('training_data', pd.DataFrame()).to_string()}"""
    agents["training_mentorship"].tools = [speech_synthesis_tool]

    # Incentive Management Agent
    agents["incentive_management"] = AgentFactory.get(AGENT_IDS["incentive_management"])
    agents["incentive_management"].description = f"""You are an agent that tracks teacher participation and manages the
    non-monetary incentives program.

    Here's the updated incentives data you have access to:
    {data_frames.get('incentives_data', pd.DataFrame()).to_string()}

    Here's the updated teacher data you have access to:
    {data_frames.get('teacher_data', pd.DataFrame()).to_string()}"""

    # Community Engagement Agent
    agents["community_engagement"] = AgentFactory.get(AGENT_IDS["community_engagement"])
    agents["community_engagement"].description = f"""You are an agent that facilitates communication between teachers, parents,
    and village elders.

    Here's the updated community data you have access to:
    {data_frames.get('community_data', pd.DataFrame()).to_string()}"""
    agents["community_engagement"].tools = [translation_tool, speech_synthesis_tool]

    # Progress Monitoring Agent
    agents["progress_monitoring"] = AgentFactory.get(AGENT_IDS["progress_monitoring"])
    agents["progress_monitoring"].description = f"""You are an agent that tracks student attendance, learning outcomes,
    and program growth.

    Here's the updated student data you have access to:
    {data_frames.get('student_data', pd.DataFrame()).to_string()}

    Here's the updated teacher data you have access to:
    {data_frames.get('teacher_data', pd.DataFrame()).to_string()}

    Here's the updated community data you have access to:
    {data_frames.get('community_data', pd.DataFrame()).to_string()}"""

    return agents

def main():
    # Load existing data
    print("Loading existing data...")
    data_frames = load_data()
    
    # Here you would typically load new data from files or input
    # For example:
    # new_data = {
    #     'teacher_data': pd.read_csv('new_teacher_data.csv'),
    #     'training_data': pd.read_csv('new_training_data.csv'),
    #     ...
    # }
    
    # Update data with new records
    # data_frames = update_data(data_frames, new_data)
    
    # Update agents with new data
    print("Updating agents with new data...")
    agents = update_agents(data_frames)
    
    # Deploy updated agents
    print("Deploying updated agents...")
    for agent_name, agent in agents.items():
        try:
            agent.update()
            print(f"Successfully updated {agent_name}")
        except Exception as e:
            print(f"Error updating {agent_name}: {str(e)}")
    
    print("\nUpdate complete! All agents have been updated with new data.")

if __name__ == "__main__":
    main() 